# Sample chatbot response

We first start with sample chat bot response without any optimization of using any special technique.

In [77]:
from typing import List, Dict, Tuple, Optional, Any
import json

In [78]:
from ollama import chat
from ollama import ChatResponse
import logging

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
)

MODEL_NAME: str = "tinyllama"


class LLM:
    def __init__(self, model_name: str) -> None:
        self.model_name = model_name

    def chat(self, messages: List[Dict[str, str]]) -> str:
        response: ChatResponse = chat(
            model=self.model_name,
            messages=messages,
        )
        return response.message.content

    def chat_with_user(
        self, user_message: str, context: Optional[List[str]] = None
    ) -> str:
        messages = [
            {
                "role": "system",
                "content": "You are a chatbot. Your job is to answer to the users query.",
            },
            {"role": "user", "content": user_message},
        ]

        if context:
            context_text = "\n".join(context)
            messages[0]["content"] += (
                f"\n\n You have additional information to improve your answer:\n{context_text}"
            )

        response: ChatResponse = chat(model=self.model_name, messages=messages)
        return response.message.content


# Sample usage.

llm = LLM(model_name=MODEL_NAME)
print(
    llm.chat_with_user(
        "My dog died few days ago. I am feeling void. What should I do to fill my void.!"
    )
)

2025-03-24 11:15:29,733 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


As a chatbot, I do not have emotions or feelings, but here are some possible responses to help you understand how to fill your void:

1. "I understand your sadness and emotional state. It's natural for humans to miss their lost pets, especially in the initial days of grief."
2. "The death of a pet can be a very traumatic event that takes time to heal. You might want to explore online resources or support groups where you can connect with others who have gone through similar experiences."
3. "Sometimes taking time off work, volunteering, or engaging in activities that bring joy and fulfillment can help ease your emotional distress."
4. "Try talking to a therapist or counselor to talk through the issues you're facing. They may be able to offer guidance on coping strategies, support groups or other resources to explore."
5. "Sending you a virtual hug or paw-print emoji is always appreciated. It can help reconnect with memories and feel less lonely during this time."

I hope these suggesti

# DEMO rank ranking algorithms for documents.

A Demo Rank Algorithm is a simplified ranking mechanism used to order items based on predefined criteria. It typically assigns scores to items based on relevance, importance, or similarity to a given query. In information retrieval, ranking algorithms like BM25, PageRank, or Cosine Similarity are commonly used to sort search results.

For example, in a cosine similarity-based ranking system, documents are ranked by computing their similarity to a query vector. The higher the similarity score, the higher the document appears in the results.

In [79]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


class DocumentRanker:
    def __init__(self, documents: List[Dict[str, str]]) -> None:
        """
        Initialize the DocumentRanker with a List of documents.
        Each document should be a dictionary with 'id' and 'content' keys.
        """
        self.documents: List[Dict[str, str]] = documents
        self.vectorizer: CountVectorizer = CountVectorizer()
        self.doc_term_matrix = self.vectorizer.fit_transform(
            [doc["problem"] for doc in documents]
        )

    def rank_documents(
        self, query: str, limit: int = 3
    ) -> List[Tuple[Dict[str, str], float]]:
        """
        Rank documents based on their relevance to the query using cosine similarity.
        :param query: The query string to compare against the documents.
        :param limit: The number of top documents to return.
        :return: A List of ranked documents with their scores.
        """
        query_vector = self.vectorizer.transform([query])
        scores = cosine_similarity(query_vector, self.doc_term_matrix).flatten()
        ranked_documents = sorted(
            zip(self.documents, scores), key=lambda x: x[1], reverse=True
        )

        # Filter out only the documents with a score greater than 0.5
        ranked_documents = [
            (doc, score) for doc, score in ranked_documents if score > 0.4
        ]
        return ranked_documents[:limit]

    def display_ranked_documents(
        self, ranked_documents: List[Tuple[Dict[str, str], float]]
    ) -> None:
        """
        Display the ranked documents in a readable format.
        :param ranked_documents: A List of tuples containing documents and their scores.
        """
        print("Ranked Documents:")
        for rank, (doc, score) in enumerate(ranked_documents, start=1):
            print(f"Rank {rank}: Document ID {doc['id']} with score {score:.4f}")
            print(f"Content: {doc['solution']}")
            print()

In [80]:
# Sample demonstration of the DocumentRanker class

documents: List[Dict[str, Any]] = []

with open("files/docs.json", "r") as json_file:
    data = json.load(json_file)
    for doc in data:
        documents.append(doc)

query = "I am feeling lost and overwhelmed. It seems to be too much to handle. How can I find my way back?"

ranker = DocumentRanker(documents)
ranked_docs = ranker.rank_documents(query)
ranker.display_ranked_documents(ranked_docs)

Ranked Documents:
Rank 1: Document ID 2 with score 0.4576
Content: Solution: Take a step back and acknowledge your feelings without judgment. Break down tasks into smaller steps and seek support from trusted people.



# Add vector semantic search 


Vector semantic search uses vector embeddings to find relevant results based on meaning rather than exact keywords.

Text is converted into high-dimensional vector representations using models like Word2Vec, BERT, or OpenAI embeddings.

A query is also transformed into a vector, and similarity (e.g., cosine similarity) is used to rank results.

## Advantages:
- Finds results based on context and meaning rather than exact matches.
- Handles synonyms and paraphrasing well.
- Supports multimodal search (text, images, audio).

## Use Cases:

- Search engines (e.g., Google, e-commerce search).
- Chatbots and AI assistants for intent matching.
- Recommendation systems (suggesting similar products, articles).

We are using Qdrant an open source vector database.

In [81]:
import logging
from collections import OrderedDict
import uuid

from qdrant_client import QdrantClient, models
from qdrant_client.models import PointStruct
from qdrant_client.http.models import VectorParams, Distance
import ollama


class SemanticEmbeddingService:
    def __init__(self, cache_size: int = 1000) -> None:
        self.embeddings_cache: OrderedDict[str, List[float]] = OrderedDict()
        self.cache_size = cache_size

    def get_embeddings(self, text: str) -> List[float]:
        if text in self.embeddings_cache:
            self.embeddings_cache.move_to_end(text)
            return self.embeddings_cache[text]
        else:
            response = ollama.embed(model="mxbai-embed-large", input=text)
            self.embeddings_cache[text] = response.embeddings[0]
            logging.info(f"Embedding for {text} is {str(response.embeddings[0])}")
            if len(self.embeddings_cache) > self.cache_size:
                self.embeddings_cache.popitem(last=False)

            return self.embeddings_cache[text]

In [82]:
# Initialize the SemanticEmbeddingService with a cache size of 1000
embedding_service = SemanticEmbeddingService(cache_size=1000)

# Example text to generate embeddings for
text = "How are you feeling today?"

# Get embeddings for the text
embeddings = embedding_service.get_embeddings(text)

# Print the embeddings
print(f"Embeddings for '{text}': {embeddings}")

2025-03-24 11:15:29,910 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-03-24 11:15:29,911 [INFO] Embedding for How are you feeling today? is [0.043695956, 0.010858177, -0.06955836, 0.009455128, -0.050337486, -0.06425655, 0.05579863, 0.03485519, 0.030004207, 0.007068535, 0.008635414, -0.0010591889, -0.023559256, -0.006996655, -0.05248184, -0.05387779, -0.013702914, -0.01057106, -0.011416469, 0.021322649, -0.05521407, 0.04312233, -0.037936006, 0.043790746, -0.04823419, 0.030281376, -0.010078121, -0.023404298, 0.0140533885, 0.044447783, -0.05962183, 0.033852488, 0.014406954, -0.0373475, 0.028444864, -0.03808326, 0.021238487, -0.03277814, 0.018745631, -0.04428547, -0.0054128533, 0.0078378795, 0.0149451075, -0.027841574, -0.0528841, -0.026624847, -0.035703022, -0.048185285, 0.01983231, -0.048971873, 0.01612873, 0.00080401707, -0.014929142, -0.016467892, 0.03597696, 0.003864972, -0.03095007, 0.056044403, -0.04840845, 0.019654213, 0.031665016, 0.006245119, -

Embeddings for 'How are you feeling today?': [0.043695956, 0.010858177, -0.06955836, 0.009455128, -0.050337486, -0.06425655, 0.05579863, 0.03485519, 0.030004207, 0.007068535, 0.008635414, -0.0010591889, -0.023559256, -0.006996655, -0.05248184, -0.05387779, -0.013702914, -0.01057106, -0.011416469, 0.021322649, -0.05521407, 0.04312233, -0.037936006, 0.043790746, -0.04823419, 0.030281376, -0.010078121, -0.023404298, 0.0140533885, 0.044447783, -0.05962183, 0.033852488, 0.014406954, -0.0373475, 0.028444864, -0.03808326, 0.021238487, -0.03277814, 0.018745631, -0.04428547, -0.0054128533, 0.0078378795, 0.0149451075, -0.027841574, -0.0528841, -0.026624847, -0.035703022, -0.048185285, 0.01983231, -0.048971873, 0.01612873, 0.00080401707, -0.014929142, -0.016467892, 0.03597696, 0.003864972, -0.03095007, 0.056044403, -0.04840845, 0.019654213, 0.031665016, 0.006245119, -0.012131748, -0.083834164, -0.018723374, 0.0024081473, -0.02346067, -0.011248264, -0.009187163, -0.018773232, -0.047691453, 0.00884

In [83]:
COLLECTION_NAME = "emotional_chatbot"


class SemanticQdrantService:
    def __init__(self, url: str) -> None:
        self.client = QdrantClient(url=url)

    def collection_exists(self, collection_name) -> bool:
        try:
            response = self.client.get_collection(collection_name)
            return response is not None
        except Exception as e:
            return False

    def create_collection(self, collection_name: str) -> None:
        print(
            f"If the collection {collection_name} does not exist, create it. {self.collection_exists(collection_name)}"
        )
        if self.collection_exists(collection_name) is False:
            self.client.create_collection(
                collection_name=collection_name,
                vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
            )

    def delete_collection(self, collection_name: str) -> None:
        if self.collection_exists(collection_name):
            self.client.delete_collection(collection_name)

    def upsert_points(self, collection_name: str, points: List[PointStruct]) -> None:
        self.client.upsert(collection_name=collection_name, points=points)

    def search(self, query_embedding: List[float], limit: int = 3) -> List[PointStruct]:
        return self.client.search(
            collection_name=COLLECTION_NAME,
            query_vector=query_embedding,
            limit=limit,
        )

In [84]:
# Initialize the service
url = "http://localhost:6333"  # Replace with your Qdrant URL
service = SemanticQdrantService(url=url)

# Create a collection if it doesn't exist
collection_name = "emotional_chatbot"
service.delete_collection(collection_name)
service.create_collection(collection_name)

2025-03-24 11:15:29,954 [INFO] HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-03-24 11:15:29,957 [INFO] HTTP Request: GET http://localhost:6333/collections/emotional_chatbot "HTTP/1.1 200 OK"
2025-03-24 11:15:29,962 [INFO] HTTP Request: DELETE http://localhost:6333/collections/emotional_chatbot "HTTP/1.1 200 OK"
2025-03-24 11:15:29,963 [INFO] HTTP Request: GET http://localhost:6333/collections/emotional_chatbot "HTTP/1.1 404 Not Found"
2025-03-24 11:15:29,964 [INFO] HTTP Request: GET http://localhost:6333/collections/emotional_chatbot "HTTP/1.1 404 Not Found"


If the collection emotional_chatbot does not exist, create it. False


2025-03-24 11:15:30,149 [INFO] HTTP Request: PUT http://localhost:6333/collections/emotional_chatbot "HTTP/1.1 200 OK"


In [85]:
# Sample usage of the SemanticQdrantService class


from qdrant_client.models import PointStruct


def use_semantic_qdrant_service():
    # Initialize the embedding service
    embedding_service = SemanticEmbeddingService(cache_size=1000)

    # Example texts to generate embeddings for
    text1 = "How are you feeling today?"
    text2 = "What is your emotional state?"

    # Get embeddings for the texts
    embedding1 = embedding_service.get_embeddings(text1)
    embedding2 = embedding_service.get_embeddings(text2)

    # Initialize the Qdrant service
    url = "http://localhost:6333"  # Replace with your Qdrant URL
    service = SemanticQdrantService(url=url)

    # Create a collection if it doesn't exist
    collection_name = "emotional_chatbot"
    service.create_collection(collection_name)

    # Upsert points into the collection
    points = [
        PointStruct(
            id=1,
            vector=embedding1,  # Use generated embedding
            payload={
                "solution": "Solution: Treat yourself as you would treat a friend. Practice self-compassion by using kind words and forgiving yourself for mistakes.",
                "problem": "Problem: Negative self-talk and self-criticism can make you feel unworthy of care and support.",
            },
        ),
    ]
    service.upsert_points(collection_name=collection_name, points=points)

    # Perform a search
    query_text = "How are you feeling today?"
    query_embedding = embedding_service.get_embeddings(query_text)
    results = service.search(query_embedding=query_embedding, limit=5)

    # Print the search results
    for result in results:
        print(f"ID: {result.id}, Score: {result.score}, Payload: {result.payload}")


# Call the function
use_semantic_qdrant_service()

2025-03-24 11:15:30,294 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-03-24 11:15:30,295 [INFO] Embedding for How are you feeling today? is [0.043695956, 0.010858177, -0.06955836, 0.009455128, -0.050337486, -0.06425655, 0.05579863, 0.03485519, 0.030004207, 0.007068535, 0.008635414, -0.0010591889, -0.023559256, -0.006996655, -0.05248184, -0.05387779, -0.013702914, -0.01057106, -0.011416469, 0.021322649, -0.05521407, 0.04312233, -0.037936006, 0.043790746, -0.04823419, 0.030281376, -0.010078121, -0.023404298, 0.0140533885, 0.044447783, -0.05962183, 0.033852488, 0.014406954, -0.0373475, 0.028444864, -0.03808326, 0.021238487, -0.03277814, 0.018745631, -0.04428547, -0.0054128533, 0.0078378795, 0.0149451075, -0.027841574, -0.0528841, -0.026624847, -0.035703022, -0.048185285, 0.01983231, -0.048971873, 0.01612873, 0.00080401707, -0.014929142, -0.016467892, 0.03597696, 0.003864972, -0.03095007, 0.056044403, -0.04840845, 0.019654213, 0.031665016, 0.006245119, -

If the collection emotional_chatbot does not exist, create it. True
ID: 1, Score: 0.99999976, Payload: {'solution': 'Solution: Treat yourself as you would treat a friend. Practice self-compassion by using kind words and forgiving yourself for mistakes.', 'problem': 'Problem: Negative self-talk and self-criticism can make you feel unworthy of care and support.'}


In [86]:
class SemanticSearchRepo:
    def __init__(
        self,
        embedding_service: SemanticEmbeddingService,
        qdrant_service: SemanticQdrantService,
    ):
        self.embedding_service = embedding_service
        self.qdrant_service = qdrant_service

    def prepare_points(
        self, texts: List[str], metadata: List[Dict]
    ) -> List[PointStruct]:
        return [
            PointStruct(
                id=str(uuid.uuid4()),
                vector=self.embedding_service.get_embeddings(text),
                payload={"text": text, **meta},
            )
            for _, (text, meta) in enumerate(zip(texts, metadata))
        ]

    async def create_collection(self, collection_name: str):
        collection_exists = self.qdrant_service.client.collection_exists(
            collection_name=collection_name
        )
        if collection_exists is False:
            self.qdrant_service.client.create_collection(
                collection_name,
                vectors_config=models.VectorParams(
                    size=1024, distance=models.Distance.COSINE
                ),
            )

    def initialize_qdrant(
        self, texts: List[str], metadata: List[Dict[str, str]]
    ) -> bool:
        points = self.prepare_points(texts, metadata)
        self.qdrant_service.upsert_points(COLLECTION_NAME, points)
        return True

    def query_text(self, query_text: str, threshold: float = 0.6) -> List[Dict]:
        query_embedding = self.embedding_service.get_embeddings(query_text)
        response = self.qdrant_service.search(query_embedding)
        logging.info(f"Query: {query_text}")
        result = []

        logging.info(f"Response: {response}")
        for data in response:
            if data.score > threshold:
                entry = {
                    "score": data.score,
                    "solution": data.payload["solution"],
                    "metadata": data.payload,
                }
                if entry not in result:
                    result.append(entry)

        print(f"The result is {result}")
        return result

In [87]:
with open("files/docs.json", "r") as json_file:
    data = json.load(json_file)
    for doc in data:
        documents.append(doc)

    # Initialize the required services
embedding_service = SemanticEmbeddingService()
qdrant_service = SemanticQdrantService(url=url)

# Create an instance of SemanticSearchRepo
semantic_repo = SemanticSearchRepo(embedding_service, qdrant_service)


def sample_usage():
    # Prepare the data
    metadata: List[Dict[str, str]] = [
        {"solution": item["solution"], "problem": item["problem"]} for item in documents
    ]
    texts: List[str] = [item["problem"] for item in metadata]
    # # Initialize Qdrant with the sample data
    semantic_repo.initialize_qdrant(texts, metadata)


sample_usage()

2025-03-24 11:15:30,526 [INFO] HTTP Request: GET http://localhost:6333 "HTTP/1.1 200 OK"
2025-03-24 11:15:30,670 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-03-24 11:15:30,672 [INFO] Embedding for Problem: My pet died and I feel empty and lost without them. is [0.024788342, -0.017987087, -0.029373357, 0.011878587, -0.033691194, -0.08373615, 0.048435986, 0.04041211, 0.049338914, 0.0044145025, 0.014437469, -0.022613045, -0.05417108, 0.016619865, -0.033240635, -0.025797607, 0.023333842, -0.018914893, -0.038682867, 0.030981438, -0.019761855, 0.033875156, -0.026498388, -0.009493013, 0.005874423, 0.0032157288, 0.01283108, -0.007623584, 0.019390684, 0.01996637, -0.024694532, 0.011526193, 0.007738205, -0.04086829, 0.016971935, -0.020198714, -0.043145467, -0.024963127, -0.027502807, -0.011596685, 0.02583234, -0.027050015, 0.06312024, -0.051842704, -0.032853115, 0.012957689, -0.0033904042, -0.014072168, 0.028817985, -0.054460905, -0.017350938, -0.0098632425,

In [ ]:
def query():
    # Query the data
    query_text = "My dog died few days ago. I am feeling void. What should I do to fill my void.!"

    threshold = 0.8
    results = semantic_repo.query_text(query_text, threshold=threshold)

    # Print the results
    print("Query Results:")
    for idx, result in enumerate(results):
        print(f"{idx + 1}. Score: {result['score']:.4f}")


query()

2025-03-24 11:15:32,106 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-03-24 11:15:32,106 [INFO] Embedding for My dog died few days ago. I am feeling void. What should I do to fill my void.! is [0.0021736494, 0.00021706684, -0.044968702, 0.023970352, -0.04997377, -0.059913654, 0.06243301, 0.03780875, 0.04485832, 0.026167184, 0.030817887, -0.0003076325, -0.028986368, 0.0019954348, -0.007606902, -0.043098677, 0.015005524, -0.03481135, -0.034606434, 0.03474081, -0.044531573, 0.06425773, -0.0503113, -0.020199018, 0.00807667, 0.0054307436, -0.025927577, -0.0070501138, 0.035536394, 0.041029643, -0.07026203, 0.030447256, -0.0016623979, -0.047431182, 0.025770621, -0.03898293, -0.029055985, -0.031049397, -0.024438223, -0.011509127, 0.009078981, -0.011448256, 0.055123486, -0.03314456, -0.024708498, -0.00187062, -0.02926191, -0.025171578, 0.02140833, -0.02450774, -0.031813577, -0.000855222, -0.020369137, -0.040434584, 0.049293015, -0.047080994, 0.01271455, 0.018

The result is [{'score': 0.8634058, 'solution': "Solution: Losing a pet is like losing a close friend, and the emptiness can be overwhelming. Allow yourself to grieve—it's natural to feel sad, lost, or even guilty. Talking to someone who understands, whether a friend, family member, or a pet loss support group, can help ease the pain.Honoring your pet’s memory might bring comfort. You could create a small tribute, like a photo album or a scrapbook, or plant a tree in their memory. Doing something positive, like donating to an animal shelter, can also be a meaningful way to keep their spirit alive.In time, finding a new routine can help you adjust. If you feel ready, volunteering at an animal shelter or eventually adopting another pet might bring joy back into your life. Grief takes time, so be patient with yourself. ", 'metadata': {'text': 'Problem: My pet died and I feel empty and lost without them.', 'solution': "Solution: Losing a pet is like losing a close friend, and the emptiness

In [89]:
# Adding vector store context to the chatbot

llm = LLM(model_name=MODEL_NAME)

query = "My dog died last week. I am feeling void. What should I do?"

context: List[str] = []

# Take out context from the demo rank
ranked_documents = ranker.rank_documents(query)
for doc, _ in ranked_documents:
    logging.info(f"Document: {doc}")
    context.append(doc["solution"])

# Take out the context from the semantic search
results = semantic_repo.query_text(query)
for idx, result in enumerate(results):
    logging.info(f"{idx} Result: {result}")
    context.append(result["solution"])

2025-03-24 11:15:32,259 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2025-03-24 11:15:32,260 [INFO] Embedding for My dog died last week. I am feeling void. What should I do? is [0.0145826535, 0.0077186613, -0.056517348, 0.014796623, -0.038969796, -0.050568048, 0.07227462, 0.019454397, 0.037362613, 0.008944393, 0.015020367, -0.016219076, -0.039840776, 0.013669304, -0.01112513, -0.037651073, 0.01636768, -0.035692044, -0.03148944, 0.04407432, -0.04462299, 0.062576525, -0.041950524, -0.023017015, 0.0043069273, 0.0014207107, -0.017223464, -0.023661422, 0.0404609, 0.052703995, -0.065286055, 0.038602106, -0.0011990649, -0.039889734, 0.022955783, -0.03912221, -0.023830224, -0.02330031, -0.042801786, -0.015670834, 0.010729797, -0.0010033182, 0.049142893, -0.048924234, -0.018204326, -0.009193855, -0.0019365564, -0.02820022, 0.02988838, -0.035307765, -0.034237556, 0.00035401547, -0.020040648, -0.03981828, 0.04181283, -0.060223248, 0.036272425, 0.045370206, -0.00071

The result is [{'score': 0.8818606, 'solution': "Solution: Losing a pet is like losing a close friend, and the emptiness can be overwhelming. Allow yourself to grieve—it's natural to feel sad, lost, or even guilty. Talking to someone who understands, whether a friend, family member, or a pet loss support group, can help ease the pain.Honoring your pet’s memory might bring comfort. You could create a small tribute, like a photo album or a scrapbook, or plant a tree in their memory. Doing something positive, like donating to an animal shelter, can also be a meaningful way to keep their spirit alive.In time, finding a new routine can help you adjust. If you feel ready, volunteering at an animal shelter or eventually adopting another pet might bring joy back into your life. Grief takes time, so be patient with yourself. ", 'metadata': {'text': 'Problem: My pet died and I feel empty and lost without them.', 'solution': "Solution: Losing a pet is like losing a close friend, and the emptiness

In [90]:
results: List[Dict[str, str]] = []

response_without_context = llm.chat_with_user(query)
logging.info(
    f"Llama response without context:\n===============================================>\n {response_without_context}"
)

results.append({"response without context": response_without_context})

response_with_context = llm.chat_with_user(query, context=context)
logging.info(
    f"Llama response with context:\n===================================================>\n {response_with_context}"
)

results.append({"response with context": response_with_context})

# Dump the results in json

with open("files/results.json", "w") as json_file:
    json.dump(results, json_file, indent=2)

2025-03-24 11:15:34,990 [INFO] HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-24 11:15:34,990 [INFO] Llama response without context:
===============================================>
 Dear User,

It's very sad to hear about your beloved pet's passing away. I'm sorry to hear this news. Here are some things you can try to cope with your emotions:

1. Take care of yourself: Try and take a break from work or social activities for the next few weeks. You need time to mourn and grieve, and to allow yourself to process your emotions.

2. Reach out to others: It's okay to feel lonely and sad in the absence of your furry friend. Reach out to friends or family members who can help you through this difficult time. You may also consider joining a dog-friendly group online, such as the Dogs Trust UK or the Barking Mad community in the US.

3. Practice self-care: Treat yourself with kindness and compassion, just like you would for yourself if it were you who was facing t

# Add memory to the application

Adding memory to the LLM can improve the emotional results. As each person has differnt levels of emotional intelligence, its good practice to train the system based on the previous interactions.